In [1]:
cd ../src

/workspace/Script/NLP/PII/src


In [2]:
# pip install deepspeed

In [3]:
# conda install -y mpi4py 

In [4]:
import os
import gc
import sys
import json
import time
import torch
import joblib
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pathlib import Path
import plotly.express as px
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [5]:
import re
import os
import gc
import math
import time
import json
import random
import numpy as np
import pandas as pd
import wandb

from pathlib import Path

import torch 
import torch.nn as nn
from torch.cuda import amp
import torch.optim as optim
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel, AutoConfig
 
from data.data_utils import to_gpu,to_np
from data.dataset import FeedbackDataset,CustomCollator
from torch.utils.data import DataLoader

from model_zoo.models import FeedbackModel,span_nms,aggregate_tokens_to_words
from metrics_loss.metrics import score_feedback,score,pii_fbeta_score_v2,compute_metrics
from transformers import get_linear_schedule_with_warmup,get_cosine_schedule_with_warmup,get_polynomial_decay_schedule_with_warmup,get_cosine_with_hard_restarts_schedule_with_warmup

from sklearn.metrics import log_loss 
from tqdm.auto import tqdm

from utils.utils import count_parameters
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [6]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# Params

In [7]:
data_path = Path(r"/database/kaggle/PII/data")
os.listdir(data_path)

['train.json',
 'mpware_mixtral8x7b_v1.1-no-i-username.json',
 'pii_dataset_fixed.csv',
 'mixtral-8x7b-v1.json',
 '.~lock.lecture2.pptx#',
 'Fake_data_1850_218.json',
 'test.json',
 'archive.zip',
 'archive',
 'pii-masking-200k.csv',
 'sample_submission.csv',
 'mpware_mixtral8x7b_v1.1.json']

In [8]:
CHECKPOINT_PATH = Path(r"/database/kaggle/PII/checkpoint")

In [13]:
df = pd.read_json(data_path/'train.json')
df.shape

(6807, 5)

In [16]:
print(df.sample(1).full_text.values[0])

Storytelling    Challenge: I was working for a PR Consultancy whose client was one of the top philanthropic  organizations in India that reaches millions of people in India. They’re known for grassroots- level interventions that strengthen communities and bring about tangible outcomes. The media  has always shown interest in the initiatives that they anchor because they are so result-oriented  and impact-focused. Their initiatives encompass a wide range of portfolios such as healthcare,  urbanisation, nutrition, art and conservation, livelihood, water and sanitation, education, social  innovation, and sports. However, it was one of those unusual times where they were unable to  gain media traction on one of the first-ever human interest story they were doing. The keyword  was ‘story’.       It was about a para-swimmer who had conquered insurmountable odds to achieve significant  milestones and accolades with the support of this organisation. Usually, PR agencies are  equipped with the 

# Data

In [17]:
from train_utils import inference_step
from types import SimpleNamespace

In [256]:
import re
import torch
import random
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

from tqdm.auto import tqdm
import re
from difflib import SequenceMatcher

import codecs
import os
from collections import Counter
from typing import Dict, List, Tuple

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from text_unidecode import unidecode
import joblib
import torch


try:
    from faker import Faker
    fake = Faker(locale = ["fr_FR","fr_CA","en_US",'en_UK','de_DE','en_GB','en_IN','it_IT','fr_BE','es_ES'])
    # Faker.seed(0)
except:
    print('No faker installed')
    
try:
    from spacy.lang.en import English
    EN_TOK = English().tokenizer
except:
    print("No spacy")


def remove_double_spaces(text):
    # Use a regular expression to replace consecutive spaces with a single space
    # cleaned_text = re.sub(r'\s{2,}', ' | ', text)
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text
    
def clean_text(text):
    text = text.replace(u'\x9d', u' ')
    text = remove_double_spaces(text)
    text = text.strip()
    return text
# ======================================================================================== #
def get_text_start_end(txt, s, search_from=0):
    txt = txt[int(search_from):]
    try:
        idx = txt.find(s)
        if idx >= 0:
            st = idx
            ed = st + len(s)
        else:
            raise ValueError('Error')
    except:
        res = [(m.start(0), m.end(0)) for m in re.finditer(s, txt)]
        if len(res):
            st, ed = res[0][0], res[0][1]
        else:
            m = SequenceMatcher(None, s, txt).get_opcodes()
            for tag, i1, i2, j1, j2 in m:
                if tag == 'replace':
                    s = s[:i1] + txt[j1:j2] + s[i2:]
                if tag == "delete":
                    s = s[:i1] + s[i2:]

            res = [(m.start(0), m.end(0)) for m in re.finditer(s, txt)]
            if len(res):
                st, ed = res[0][0], res[0][1]
            else:
                idx = txt.find(s)
                if idx >= 0:
                    st = idx
                    ed = st + len(s)
                else:
                    st, ed = 0, 0
    return st + search_from, ed + search_from
    
def get_offset_mapping(full_text, tokens):
    offset_mapping = []

    current_offset = 0
    for token in tokens:
        start, end = get_text_start_end(full_text, token, search_from=current_offset)
        offset_mapping.append((start, end))
        current_offset = end

    return offset_mapping
    
def tokenize_with_spacy(text,tok=EN_TOK):
    tokenized_text = tok(text)
    tokens = [token.text for token in tokenized_text]
    offset_mapping = [(token.idx,token.idx+len(token)) for token in tokenized_text]
    return {'tokens': tokens, 'offset_mapping': offset_mapping}


def DemoText(text,tokenizer):
    
    
    text = clean_text(text.replace("\n\n"," | ").replace("\n"," [BR] "))
    
    sp_tokens = tokenize_with_spacy(text)
    sp_offset_mapping = get_offset_mapping(text, sp_tokens['tokens'])
#     sp_offset_mapping = sp_tokens['offset_mapping']
    
    hf_tokens = tokenizer(text, return_offsets_mapping=True)
    input_ids = torch.LongTensor(hf_tokens['input_ids'])
    attention_mask = torch.LongTensor(hf_tokens['attention_mask'])
    hf_offset_mapping = np.array(hf_tokens['offset_mapping'])
    
    
    num_tokens = len(input_ids)

    # token slices of words
    woff = np.array(sp_offset_mapping)
    toff = np.array(hf_offset_mapping)
    wx1, wx2 = woff.T
    tx1, tx2 = toff.T
    ix1 = np.maximum(wx1[..., None], tx1[None, ...])
    ix2 = np.minimum(wx2[..., None], tx2[None, ...])
    ux1 = np.minimum(wx1[..., None], tx1[None, ...])
    ux2 = np.maximum(wx2[..., None], tx2[None, ...])
    ious = (ix2 - ix1).clip(min=0) / (ux2 - ux1)
#         assert (ious > 0).any(-1).all()

    word_boxes = []
    for i,row in enumerate(ious):
        inds = row.nonzero()[0]
        try:
            word_boxes.append([inds[0], 0, inds[-1] + 1, 1])
        except:
            word_boxes.append([-100, 0, -99, 1])
#                 err.append(i)

    word_boxes = torch.FloatTensor(word_boxes)
    
    return dict(word_boxes=word_boxes,text=text,
                sp_tokens=sp_tokens,sp_offset_mapping=sp_offset_mapping,
                input_ids = input_ids.unsqueeze(0),hf_offset_mapping=hf_offset_mapping,
                attention_mask = attention_mask.unsqueeze(0)
               )

def reverse_transformation(text):
    # Replace ' | ' with double newlines
    text = text.replace(" | ", "\n\n")
    # Replace ' [BR] ' with single newlines
    text = text.replace(" [BR] ", "\n")
    return text

In [117]:
model_name = 'microsoft/deberta-v3-xsmall'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [172]:
df

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [178]:
idx = random.choice(df[df.has_label==1].index)
full_text_ds = df.iloc[idx]['full_text']
tokens_ds = df.iloc[idx]['tokens']
labels_ds = df.iloc[idx]['labels']
len(labels_ds),len(tokens_ds)

(873, 873)

In [258]:
full_text_ds = """Name: Emily Watson
Date of Birth: April 15, 1987
Address: 123 Maple Street, Apt 4B, Springfield, IL, 62704
Phone Number: (217) 555-0198
Email: ewatson87@example.com

Occupation: Graphic Designer at Creative Minds Studio

Education:

Bachelor of Fine Arts (BFA) in Graphic Design, University of Illinois at Urbana-Champaign, 2009
Master of Fine Arts (MFA) in Visual Communication, School of the Art Institute of Chicago, 2011
Professional Experience:

Creative Minds Studio (2012-Present)

Lead Graphic Designer: Leading a team of designers in creating visual content for various clients including branding, advertisements, and web design.
Key Projects: Rebranding for local businesses, social media campaigns, and website redesign for a mid-sized tech company.
Freelance Designer (2011-2012)

Worked with various clients on projects ranging from logo design to full marketing campaigns. Developed strong client relationships and a diverse portfolio.
Skills:

Proficient in Adobe Creative Suite (Photoshop, Illustrator, InDesign)
Strong understanding of UI/UX principles
HTML/CSS basics
Excellent communication and teamwork skills
Personal Interests:

Avid painter and member of the local art club.
Volunteer at Springfield Animal Shelter, helping with graphic design for their promotional materials.
Enjoys hiking and nature photography.
Family:

Spouse: Michael Watson, high school teacher
Children: Oliver Watson (8 years old), Sophie Watson (5 years old)
Social Media:

LinkedIn: linkedin.com/in/ewatson87
Instagram: @emwatsonart
Notable Achievements:

Featured in "Top 50 Designers Under 40" by Design World Magazine, 2020.
Awarded "Best Local Designer" by Springfield Arts Council, 2019.
Speaker at the 2021 National Design Conference on "The Future of Graphic Design in Digital Media."
Community Involvement:

Organizer of the annual Springfield Art Fair, a community event showcasing local artists.
Guest lecturer at Springfield Community College on graphic design trends and career paths.
Emily Watson is a dedicated and talented graphic designer with over a decade of experience in the field. Her passion for design is evident in her work and her commitment to helping others through her volunteer efforts and community involvement. She continues to inspire with her creativity and dedication to the arts."""

In [259]:
dic = DemoText(full_text_ds,tokenizer)

In [260]:
dic['text']

'Name: Emily Watson [BR] Date of Birth: April 15, 1987 [BR] Address: 123 Maple Street, Apt 4B, Springfield, IL, 62704 [BR] Phone Number: (217) 555-0198 [BR] Email: ewatson87@example.com | Occupation: Graphic Designer at Creative Minds Studio | Education: | Bachelor of Fine Arts (BFA) in Graphic Design, University of Illinois at Urbana-Champaign, 2009 [BR] Master of Fine Arts (MFA) in Visual Communication, School of the Art Institute of Chicago, 2011 [BR] Professional Experience: | Creative Minds Studio (2012-Present) | Lead Graphic Designer: Leading a team of designers in creating visual content for various clients including branding, advertisements, and web design. [BR] Key Projects: Rebranding for local businesses, social media campaigns, and website redesign for a mid-sized tech company. [BR] Freelance Designer (2011-2012) | Worked with various clients on projects ranging from logo design to full marketing campaigns. Developed strong client relationships and a diverse portfolio. [BR

In [261]:
dic['word_boxes'].shape

torch.Size([503, 4])

In [225]:
len(dic['sp_offset_mapping']),dic['input_ids'].shape,len(dic['hf_offset_mapping']),len(dic['input_ids'])

(397, torch.Size([1, 407]), 407, 1)

In [226]:
print(" ".join(tokenizer.convert_ids_to_tokens(dic['input_ids'][0])))

[CLS] ▁Jessica ▁Thompson ▁had ▁always ▁felt ▁a ▁deep ▁connection ▁to ▁the ▁ocean . ▁Born ▁on ▁June ▁15 , ▁1985 , ▁in ▁San ▁Diego , ▁California , ▁she ▁grew ▁up ▁just ▁a ▁few ▁miles ▁from ▁the ▁beach . ▁Her ▁parents , ▁Robert ▁and ▁Susan ▁Thompson , ▁encouraged ▁her ▁love ▁for ▁the ▁sea . ▁Robert , ▁a ▁marine ▁biologist , ▁and ▁Susan , ▁an ▁environmental ▁lawyer , ▁often ▁took ▁young ▁Jessica ▁on ▁boat ▁trips , ▁instilling ▁in ▁her ▁a ▁passion ▁for ▁marine ▁conservation . ▁Jessica ▁attended ▁Pacific ▁High ▁School , ▁where ▁she ▁excelled ▁in ▁science ▁and ▁mathematics . ▁After ▁graduating ▁in ▁2003 , ▁she ▁pursued ▁a ▁degree ▁in ▁Marine ▁Biology ▁at ▁the ▁University ▁of ▁California , ▁Santa ▁Barbara , ▁graduating ▁with ▁honors ▁in ▁2007 . ▁During ▁her ▁college ▁years , ▁Jessica ▁interned ▁at ▁the ▁Monterey ▁Bay ▁Aquarium , ▁where ▁she ▁conducted ▁research ▁on ▁the ▁migratory ▁patterns ▁of ▁sea ▁turtles . ▁In ▁2009 , ▁Jessica ▁moved ▁to ▁Miami , ▁Florida , ▁to ▁work ▁with ▁the ▁Ocean ▁Con

In [227]:
dic['word_boxes'][:20]

tensor([[ 1.,  0.,  2.,  1.],
        [ 2.,  0.,  3.,  1.],
        [ 3.,  0.,  4.,  1.],
        [ 4.,  0.,  5.,  1.],
        [ 5.,  0.,  6.,  1.],
        [ 6.,  0.,  7.,  1.],
        [ 7.,  0.,  8.,  1.],
        [ 8.,  0.,  9.,  1.],
        [ 9.,  0., 10.,  1.],
        [10.,  0., 11.,  1.],
        [11.,  0., 12.,  1.],
        [12.,  0., 13.,  1.],
        [13.,  0., 14.,  1.],
        [14.,  0., 15.,  1.],
        [15.,  0., 16.,  1.],
        [16.,  0., 17.,  1.],
        [17.,  0., 18.,  1.],
        [18.,  0., 19.,  1.],
        [19.,  0., 20.,  1.],
        [20.,  0., 21.,  1.]])

In [294]:
FOLD_NAME = "fold_msk_5_seed_42"
model_name = "deberta-v3-large"
exp_name = "2024-04-06--dv3l_cp_nboard_add_00_rep00_4608"
folder = str(CHECKPOINT_PATH/Path(fr'{FOLD_NAME}/{model_name}/{exp_name}')) 
os.listdir(folder)

['fold_0 _epoch_3 _step_5499 _valid_loss_0.0039 _f5_micro_0.9980 _train_loss_0.0024.pth',
 'fold_1 _epoch_4 _step_7328 _valid_loss_0.0042 _f5_micro_0.9979 _train_loss_0.0015.pth',
 'tokenizer',
 'params.json',
 'stack_oof.gzip',
 'config.pth',
 'oof.csv',
 'fold_3 _epoch_4 _step_7320 _valid_loss_0.0071 _f5_micro_0.9984 _train_loss_0.0015.pth',
 'fold_4 _epoch_7 _step_7326 _valid_loss_0.0055 _f5_micro_0.9983 _train_loss_0.0002.pth',
 'oof_blend_min_O_2024-04-06--dv3l_cp_nboard_add_00_rep00_4608---2024-02-04--vsub1.csv',
 'fold_2 _epoch_5 _step_7333 _valid_loss_0.0025 _f5_micro_0.9982 _train_loss_0.0008.pth']

In [295]:
# ==== Loading Args =========== #
f = open(f'{folder}/params.json')
args = json.load(f)
args = SimpleNamespace(**args)
args.model['pretrained_tokenizer'] = f"{folder}/tokenizer"
args.model['model_params']['config_path'] = f"{folder}/config.pth"
args.model['pretrained_weights'] = None
args.model["model_params"]['pretrained_path'] = None
args.device = 0
device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")

In [296]:
checkpoints = [x.as_posix() for x in (Path(folder)).glob("*.pth") if f"config" not in x.as_posix()]
checkpoints

['/database/kaggle/PII/checkpoint/fold_msk_5_seed_42/deberta-v3-large/2024-04-06--dv3l_cp_nboard_add_00_rep00_4608/fold_0 _epoch_3 _step_5499 _valid_loss_0.0039 _f5_micro_0.9980 _train_loss_0.0024.pth',
 '/database/kaggle/PII/checkpoint/fold_msk_5_seed_42/deberta-v3-large/2024-04-06--dv3l_cp_nboard_add_00_rep00_4608/fold_1 _epoch_4 _step_7328 _valid_loss_0.0042 _f5_micro_0.9979 _train_loss_0.0015.pth',
 '/database/kaggle/PII/checkpoint/fold_msk_5_seed_42/deberta-v3-large/2024-04-06--dv3l_cp_nboard_add_00_rep00_4608/fold_3 _epoch_4 _step_7320 _valid_loss_0.0071 _f5_micro_0.9984 _train_loss_0.0015.pth',
 '/database/kaggle/PII/checkpoint/fold_msk_5_seed_42/deberta-v3-large/2024-04-06--dv3l_cp_nboard_add_00_rep00_4608/fold_4 _epoch_7 _step_7326 _valid_loss_0.0055 _f5_micro_0.9983 _train_loss_0.0002.pth',
 '/database/kaggle/PII/checkpoint/fold_msk_5_seed_42/deberta-v3-large/2024-04-06--dv3l_cp_nboard_add_00_rep00_4608/fold_2 _epoch_5 _step_7333 _valid_loss_0.0025 _f5_micro_0.9982 _train_los

In [297]:
net = FeedbackModel(**args.model["model_params"])
net.load_state_dict(torch.load(checkpoints[0], map_location=lambda storage, loc: storage))
net = net.to(device)
net.eval()

FeedbackModel(
  (backbone): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
  

In [298]:
preds = net(to_gpu(dic,device))['pred'].softmax(-1).detach().cpu().to(torch.float32)
s,i = preds.max(-1)

In [299]:
pred_df = pd.DataFrame({
                                    "token" : dic['sp_tokens']['tokens'],
                                    "token_id":np.arange(len(s)),
                                    "label" : i.numpy() ,
                                    "score" : np.float32(s.numpy()) ,
    #                                  "o_score":predictions[:,-1].numpy()
                                    })

In [300]:
pred_df

,token,token_id,label,score
0,Name,0,7,0.9997
1,:,1,7,0.9999
2,Emily,2,0,0.9938
3,Watson,3,0,0.9953
4,[,4,7,0.9999
...,...,...,...,...
498,dedication,498,7,0.9999
499,to,499,7,0.9999
500,the,500,7,0.9999
501,arts,501,7,0.9999


In [301]:
pred_df["label_next_e_prev"] = pred_df['label'].transform(lambda x: (x.shift(1)==x.shift(-1))*1)
pred_df["label_next"] = pred_df['label'].transform(lambda x: x.shift(1))
pred_df["label_next_e_prev"] = ((pred_df["label_next_e_prev"]==1) & (pred_df["label_next"]==6))*1
pred_df["score_next"] = pred_df['score'].transform(lambda x: x.shift(1))
pred_df.loc[pred_df["label_next_e_prev"]==1,"label"] = pred_df.loc[pred_df["label_next_e_prev"]==1,"label_next"]
pred_df.loc[pred_df["label_next_e_prev"]==1,"score"] = pred_df.loc[pred_df["label_next_e_prev"]==1,"score_next"]

In [302]:
threshold = 0.15
if threshold>0:
    pred_df = pred_df[(pred_df.label!=7) & ((pred_df.score>threshold))].reset_index(drop=True)

In [303]:
LABEL2TYPE = ('@NAMES','@EMAIL','@USERNAME','@ID_NUM', '@PHONE_NUM','@URL_PERSONAL','@STREET_ADDRESS','O')
TYPE2LABEL = {t: l for l, t in enumerate(LABEL2TYPE)}
LABEL2TYPE = {l: t for l, t in enumerate(LABEL2TYPE)}

In [304]:
pred_df['label'] = pred_df['label'].map(LABEL2TYPE)

In [305]:
pred_df

,token,token_id,label,score,label_next_e_prev,label_next,score_next
0,Emily,2,@NAMES,0.9938,0,7.0000,0.9999
1,Watson,3,@NAMES,0.9953,0,0.0000,0.9938
2,123,20,@STREET_ADDRESS,0.9992,0,7.0000,0.9999
3,Maple,21,@STREET_ADDRESS,0.9992,1,6.0000,0.9992
4,Street,22,@STREET_ADDRESS,0.9995,1,6.0000,0.9995
5,",",23,@STREET_ADDRESS,0.9996,1,6.0000,0.9996
6,Apt,24,@STREET_ADDRESS,0.9983,1,6.0000,0.9983
7,4B,25,@STREET_ADDRESS,0.9987,1,6.0000,0.9987
8,",",26,@STREET_ADDRESS,0.9991,1,6.0000,0.9991
9,Springfield,27,@STREET_ADDRESS,0.9934,1,6.0000,0.9934


In [306]:
dic['sp_offset_mapping'][3]

(12, 18)

In [307]:
dic['text'][15:19]

'son '

In [308]:
def replace_text_elements(full_text, text_elements, text_elements_replaced, offset_mapping):
    # Convert full_text to a list of characters to handle replacements
    full_text_list = list(full_text)
    offset_adjustment = 0

    for i, (start, end) in enumerate(offset_mapping):
        replacement = text_elements_replaced[i]

        # Calculate the current positions considering the offset adjustment
        start += offset_adjustment
        end += offset_adjustment

        # Replace the text in full_text_list
        full_text_list[start:end] = list(replacement)

        # Calculate the length difference caused by the replacement
        length_difference = len(replacement) - (end - start)
        
        # Update the offset adjustment for subsequent replacements
        offset_adjustment += length_difference

    # Join the list back into a string
    return ''.join(full_text_list)

# Example usage
full_text = "Hello world! This is a test."
text_elements = ["world", "test"]
text_elements_replaced = ["universe", "example"]
offset_mapping = [(6, 11), (20, 24)]  # The positions of "world" and "test" in full_text

result = replace_text_elements(full_text, text_elements, text_elements_replaced, offset_mapping)
print(result)  # Output: "Hello universe! This is a example."


Hello universe! This isexampleest.


In [309]:
full_text = dic['text']
text_elements =  pred_df.token.values.tolist()
text_elements_replaced = pred_df.label.values.tolist()
offset_mapping = [dic['sp_offset_mapping'][i] for i in pred_df.token_id.values.tolist()]

In [310]:
print(replace_text_elements(full_text, text_elements, text_elements_replaced, offset_mapping))

Name: @NAMES @NAMES [BR] Date of Birth: April 15, 1987 [BR] Address: @STREET_ADDRESS @STREET_ADDRESS @STREET_ADDRESS@STREET_ADDRESS @STREET_ADDRESS @STREET_ADDRESS@STREET_ADDRESS @STREET_ADDRESS@STREET_ADDRESS @STREET_ADDRESS@STREET_ADDRESS @STREET_ADDRESS [BR] Phone Number: @PHONE_NUM@PHONE_NUM@PHONE_NUM @PHONE_NUM@PHONE_NUM@PHONE_NUM [BR] Email: @EMAIL | Occupation: Graphic Designer at Creative Minds Studio | Education: | Bachelor of Fine Arts (BFA) in Graphic Design, University of Illinois at Urbana-Champaign, 2009 [BR] Master of Fine Arts (MFA) in Visual Communication, School of the Art Institute of Chicago, 2011 [BR] Professional Experience: | Creative Minds Studio (2012-Present) | Lead Graphic Designer: Leading a team of designers in creating visual content for various clients including branding, advertisements, and web design. [BR] Key Projects: Rebranding for local businesses, social media campaigns, and website redesign for a mid-sized tech company. [BR] Freelance Designer (20

In [311]:
print(reverse_transformation(replace_text_elements(full_text, text_elements, 
                                                   text_elements_replaced, offset_mapping)))

Name: @NAMES @NAMES
Date of Birth: April 15, 1987
Address: @STREET_ADDRESS @STREET_ADDRESS @STREET_ADDRESS@STREET_ADDRESS @STREET_ADDRESS @STREET_ADDRESS@STREET_ADDRESS @STREET_ADDRESS@STREET_ADDRESS @STREET_ADDRESS@STREET_ADDRESS @STREET_ADDRESS
Phone Number: @PHONE_NUM@PHONE_NUM@PHONE_NUM @PHONE_NUM@PHONE_NUM@PHONE_NUM
Email: @EMAIL

Occupation: Graphic Designer at Creative Minds Studio

Education:

Bachelor of Fine Arts (BFA) in Graphic Design, University of Illinois at Urbana-Champaign, 2009
Master of Fine Arts (MFA) in Visual Communication, School of the Art Institute of Chicago, 2011
Professional Experience:

Creative Minds Studio (2012-Present)

Lead Graphic Designer: Leading a team of designers in creating visual content for various clients including branding, advertisements, and web design.
Key Projects: Rebranding for local businesses, social media campaigns, and website redesign for a mid-sized tech company.
Freelance Designer (2011-2012)

Worked with various clients on proj

In [312]:
def replace_successive_placeholders(text):
    placeholders = ['@NAMES', '@EMAIL', '@USERNAME', '@ID_NUM', '@PHONE_NUM', '@URL_PERSONAL', '@STREET_ADDRESS']
    
    for placeholder in placeholders:
        pattern = re.compile(r'(' + re.escape(placeholder) + r')(\s*\1)+')
        text = pattern.sub(placeholder, text)
    
    return text

In [315]:
text_anonymized = replace_successive_placeholders(reverse_transformation(replace_text_elements(full_text, text_elements, 
                                                   text_elements_replaced, offset_mapping)))
print(text_anonymized)

Name: @NAMES
Date of Birth: April 15, 1987
Address: @STREET_ADDRESS
Phone Number: @PHONE_NUM
Email: @EMAIL

Occupation: Graphic Designer at Creative Minds Studio

Education:

Bachelor of Fine Arts (BFA) in Graphic Design, University of Illinois at Urbana-Champaign, 2009
Master of Fine Arts (MFA) in Visual Communication, School of the Art Institute of Chicago, 2011
Professional Experience:

Creative Minds Studio (2012-Present)

Lead Graphic Designer: Leading a team of designers in creating visual content for various clients including branding, advertisements, and web design.
Key Projects: Rebranding for local businesses, social media campaigns, and website redesign for a mid-sized tech company.
Freelance Designer (2011-2012)

Worked with various clients on projects ranging from logo design to full marketing campaigns. Developed strong client relationships and a diverse portfolio.
Skills:

Proficient in Adobe Creative Suite (Photoshop, Illustrator, InDesign)
Strong understanding of UI/UX

In [326]:
import spacy
from spacy import displacy
from pylab import cm, matplotlib
import os

colors = {
            '@NAME_STUDENT': '#8000ff',
            '@EMAIL': '#2b7ff6',
            '@USERNAME': '#2adddd',
            '@ID_NUM': '#80ffb4',
            '@PHONE_NUM': 'd4dd80',
            '@URL_PERSONAL': '#ff8042',
            '@STREET_ADDRESS': '#ff0000'
         }


def visualize(full_text,offset_mapping):
    
    ents = []
    for offset in offset_mapping:
        ents.append({
                        'start': int(offset["start_offset"]), 
                         'end': int(offset["end_offset"]), 
                         'label': str(offset["placeholder"]) #+ ' - ' + str(row['discourse_effectiveness'])
                    })
    
    print(ents)
    doc2 = {
        "text": full_text,
        "ents": ents,
#         "title": "idx"
    }

    options = {"ents": list(colors.keys()), "colors": colors}
    displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)

In [327]:
import re

def find_placeholders_offsets(text):
    placeholders = ['@NAMES', '@EMAIL', '@USERNAME', '@ID_NUM', '@PHONE_NUM', '@URL_PERSONAL', '@STREET_ADDRESS']
    offsets = []
    
    for placeholder in placeholders:
        for match in re.finditer(re.escape(placeholder), text):
            offsets.append({
                'placeholder': placeholder,
                'start_offset': match.start(),
                'end_offset': match.end()
            })
    
    return offsets


In [328]:
offsets = find_placeholders_offsets(text_anonymized)

In [329]:
offsets

[{'placeholder': '@NAMES', 'start_offset': 6, 'end_offset': 12},
 {'placeholder': '@NAMES', 'start_offset': 1297, 'end_offset': 1303},
 {'placeholder': '@NAMES', 'start_offset': 1335, 'end_offset': 1341},
 {'placeholder': '@NAMES', 'start_offset': 1357, 'end_offset': 1363},
 {'placeholder': '@NAMES', 'start_offset': 1902, 'end_offset': 1908},
 {'placeholder': '@EMAIL', 'start_offset': 100, 'end_offset': 106},
 {'placeholder': '@USERNAME', 'start_offset': 1428, 'end_offset': 1437},
 {'placeholder': '@PHONE_NUM', 'start_offset': 82, 'end_offset': 92},
 {'placeholder': '@URL_PERSONAL', 'start_offset': 1403, 'end_offset': 1416},
 {'placeholder': '@STREET_ADDRESS', 'start_offset': 52, 'end_offset': 67}]

In [330]:
visualize(text_anonymized,offsets)

[{'start': 6, 'end': 12, 'label': '@NAMES'}, {'start': 1297, 'end': 1303, 'label': '@NAMES'}, {'start': 1335, 'end': 1341, 'label': '@NAMES'}, {'start': 1357, 'end': 1363, 'label': '@NAMES'}, {'start': 1902, 'end': 1908, 'label': '@NAMES'}, {'start': 100, 'end': 106, 'label': '@EMAIL'}, {'start': 1428, 'end': 1437, 'label': '@USERNAME'}, {'start': 82, 'end': 92, 'label': '@PHONE_NUM'}, {'start': 1403, 'end': 1416, 'label': '@URL_PERSONAL'}, {'start': 52, 'end': 67, 'label': '@STREET_ADDRESS'}]


In [331]:
import re
import spacy
from spacy import displacy

# Define colors for placeholders
colors = {
    '@NAMES': '#8000ff',
    '@EMAIL': '#2b7ff6',
    '@USERNAME': '#2adddd',
    '@ID_NUM': '#80ffb4',
    '@PHONE_NUM': '#d4dd80',
    '@URL_PERSONAL': '#ff8042',
    '@STREET_ADDRESS': '#ff0000'
}

def find_placeholders_offsets(text):
    placeholders = ['@NAMES', '@EMAIL', '@USERNAME', '@ID_NUM', '@PHONE_NUM', '@URL_PERSONAL', '@STREET_ADDRESS']
    offsets = []
    
    for placeholder in placeholders:
        for match in re.finditer(re.escape(placeholder), text):
            offsets.append({
                'placeholder': placeholder,
                'start_offset': match.start(),
                'end_offset': match.end()
            })
    
    return offsets

def visualize(full_text, offset_mapping):
    ents = []
    for offset in offset_mapping:
        ents.append({
            'start': int(offset["start_offset"]), 
            'end': int(offset["end_offset"]), 
            'label': offset["placeholder"]  # Keeping the label here for displacy, but it won't be shown in output
        })
    
    doc2 = {
        "text": full_text,
        "ents": ents,
    }

    options = {"ents": list(colors.keys()), "colors": colors}
    displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)


In [332]:
offsets = find_placeholders_offsets(text_anonymized)
visualize(text_anonymized, offsets)